# Data Collection

#### I used udacity generated data set  to train my model.
* Images were collected from all three cameras with following operations:-
    * Angle Adjustments of 0.22 were made to left(+.22) and Right Images(-.22)
    * Followed By fliping every image vertically and adjusting angles accordingly 
    * All these data collection was done using generators to imporve memory utilization

In [6]:
import os
import csv
from sklearn.utils import shuffle


samples = []
with open('data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        samples.append(line)

from sklearn.model_selection import train_test_split
train_samples, validation_samples = train_test_split(samples, test_size=0.2)


import cv2
import numpy as np
import sklearn

center_angles=[]
right_angles=[]
left_angles=[]
center_images=[]
right_images=[]
left_images=[]

def generator(images, batch_size=128):
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            images = []
            angles = []
            for batch_sample in batch_samples:
                filename=batch_sample[0].split('/')[-1]
                current_path='data/IMG/'+filename
                name = current_path
                image = cv2.imread(name,cv2.IMREAD_COLOR)
                image=cv2.cvtColor(image,cv2.COLOR_BGR2YUV)
                angle = float(batch_sample[3])
                images.append(image)
                angles.append(angle)
                images.append(cv2.flip(image,1))
                angles.append(-1*angle)
                #left image data with angle
                filename=batch_sample[1].split('/')[-1]
                current_path='data/IMG/'+filename
                name = current_path
                image = cv2.imread(name,cv2.IMREAD_COLOR)
                image=cv2.cvtColor(image,cv2.COLOR_BGR2YUV)
                angle = float(batch_sample[3])
                images.append(image)
                angles.append(angle+0.22)
                images.append(cv2.flip(image,1))
                angles.append(-1*(angle+0.22))
                #Right image data
                filename=batch_sample[2].split('/')[-1]
                current_path='data/IMG/'+filename
                name = current_path
                image = cv2.imread(name,cv2.IMREAD_COLOR)
                image=cv2.cvtColor(image,cv2.COLOR_BGR2YUV)
                angle = float(batch_sample[3])
                images.append(image)
                angles.append(angle-0.22)
                images.append(cv2.flip(image,1))
                angles.append(-1*(angle-0.22))
             

                

            # trim image to only see section with road
            X_train = np.array(images)
            y_train = np.array(angles)
            yield sklearn.utils.shuffle(X_train, y_train)
            yield sklearn.utils.shuffle(X_train, y_train)
             # compile and train the model using the generator function

train_generator = generator(train_samples, batch_size=128)
validation_generator = generator(validation_samples, batch_size=128)


 
# Data Learing Model

* In this model I first adjusted the images to range 0 to 1 using lamba function in keras.
* Then I used 2 2D-Convolution neural networks of kernal 5X5 and filters 3 and 18 with Maxpooling function of 2 stides.
* This was then followed by 3X3 Kernals with filiter 36 and 64 along with Maxpooling function
* After performing this I used Dropout function to remove noise in the function
* This was followed by flatten function and further followed by Dense function of 100,80 and 1
* After Creating the model I used generators  models for execution.
##### Note:- Learning Rate= 0.0008, Losses =0.0208, Validation Losses=0.0221

In [16]:
from keras.models import Sequential,Model
from keras.layers import Flatten, Dense, Lambda, Cropping2D, Dropout
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D
from keras import optimizers
model=Sequential()
model.add(Lambda(lambda x: x/127.5 - 1,
        input_shape=(160, 320, 3)))
model.add(Cropping2D(cropping=((70,20), (0,0))))
model.add(Convolution2D(3,5,5,activation='relu'))
model.add(MaxPooling2D((2,2),strides=(2,2)))
model.add(Convolution2D(18,5,5,activation='relu'))
model.add(MaxPooling2D((2,2),strides=(2,2)))
model.add(Convolution2D(36,3,3,activation='relu'))
model.add(MaxPooling2D())
model.add(Convolution2D(64,3,3,activation='relu'))
model.add(MaxPooling2D())
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(100))
model.add(Dense(80))
model.add(Dense(1))
#Setting the learning rate for the model
adam=optimizers.adam(lr=0.0008)
model.compile(loss='mse',optimizer=adam)
model.fit_generator(train_generator, samples_per_epoch=len(train_samples), validation_data=validation_generator,nb_val_samples=len(validation_samples), nb_epoch=20)
model.save('model.h5')
model.save('model.json')

Epoch 1/20
6144/6428 [===========================>..] - ETA: 0s - loss: 0.0527

/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/keras/engine/training.py:1569: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


6912/6428 [================================] - 18s - loss: 0.0503 - val_loss: 0.0342
Epoch 2/20
6912/6428 [================================] - 15s - loss: 0.0519 - val_loss: 0.0319
Epoch 3/20
6912/6428 [================================] - 16s - loss: 0.0444 - val_loss: 0.0220
Epoch 4/20
6912/6428 [================================] - 16s - loss: 0.0275 - val_loss: 0.0275
Epoch 5/20
6912/6428 [================================] - 16s - loss: 0.0296 - val_loss: 0.0245
Epoch 6/20
6912/6428 [================================] - 16s - loss: 0.0245 - val_loss: 0.0471
Epoch 7/20
6912/6428 [================================] - 16s - loss: 0.0209 - val_loss: 0.0498
Epoch 8/20
6912/6428 [================================] - 16s - loss: 0.0285 - val_loss: 0.0327
Epoch 9/20
6912/6428 [================================] - 16s - loss: 0.0287 - val_loss: 0.0352
Epoch 10/20
6576/6428 [==============================] - 16s - loss: 0.0270 - val_loss: 0.0144
Epoch 11/20
6912/6428 [=============================